# Tests Nov 4

In [20]:
import pandas as pd
import numpy as np
import os

In [2]:
%load_ext autoreload
%autoreload 2

In [57]:
from invesscience.joanna_merge import get_training_data 

In [58]:
companies = get_training_data(reference="a")

            id        normalized_name   category_code founded_at  \
3132  c:169096          tyro payments        software 2003-02-03   
640     c:3299               vivisimo        software        NaT   
1374   c:20286             terrafugia  transportation 2006-05-01   
2221     c:425                dropbox        software 2007-06-01   
427    c:23963  widevine technologies     games_video 1998-01-01   

                                            description closed_at  \
3132  software   Australian Merchant EFTPOS Payments...       NaN   
640   software   info optimization, enterprise searc...       NaN   
1374  transportation   Roadable Airplane Terrafugia ...       NaN   
2221  software Dropbox operates a free file hosting ...       NaN   
427   games_video   Enduring Entertainment Widevine ...       NaN   

             exit            exit_date country_code state_code date_series_a  \
3132      no exit                  NaN          AUS        NaN    2005-04-01   
640   acquisitio

In [53]:
companies.head()

,id,normalized_name,category_code,founded_at,description,closed_at,exit,exit_date,country_code,state_code,...,top_20,top_50,top_5_bool,top_20_bool,top_50_bool,n_female_founders,female_ratio,mean_comp_founded_ever,mean_comp_founded_before,target
0,c:1349,move networks,software,NaT,software Move Networks is a streaming tech pla...,NaN,acquisition,1987-06-16 00:00:00,USA,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,c:4584,modern feed,games_video,2007-01-01,games_video [Modern Feed](http://www.moder...,NaN,acquisition,1987-06-16 00:00:00,USA,CA,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1
2,c:16612,micron technology,semiconductor,1978-01-01,semiconductor Semiconductor Applications Mic...,NaN,ipo,1990-01-02 00:00:00,USA,ID,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,c:28,cisco,network_hosting,1984-01-01,network_hosting Cisco designs and sells ha...,NaN,ipo,1990-02-23 00:00:00,USA,CA,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
4,c:29,yahoo,web,1994-01-01,web News and On-line directory Yahoo was fou...,NaN,ipo,1996-04-12 00:00:00,USA,CA,...,0.0,0.2,1.0,0.0,1.0,0.0,0.0,1.2,0.0,1


In [54]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3241 entries, 0 to 3240
Data columns (total 39 columns):
id                           3241 non-null object
normalized_name              3241 non-null object
category_code                3115 non-null object
founded_at                   2642 non-null datetime64[ns]
description                  3239 non-null object
closed_at                    360 non-null object
exit                         3241 non-null object
exit_date                    797 non-null object
country_code                 3118 non-null object
state_code                   2241 non-null object
date_series_a                3241 non-null datetime64[ns]
participants_a               3241 non-null int64
raised_amount_usd_a          3241 non-null float64
timediff_founded_series_a    2642 non-null float64
rounds_before_a              3241 non-null float64
raised_before_a              3165 non-null float64
participants_before_a        3165 non-null float64
phd                      

In [55]:
companies.columns

Index(['id', 'normalized_name', 'category_code', 'founded_at', 'description',
       'closed_at', 'exit', 'exit_date', 'country_code', 'state_code',
       'date_series_a', 'participants_a', 'raised_amount_usd_a',
       'timediff_founded_series_a', 'rounds_before_a', 'raised_before_a',
       'participants_before_a', 'phd', 'MBA', 'cs', 'graduate', 'undergrad',
       'professional', 'degree_count', 'MBA_bool', 'cs_bool', 'phd_bool',
       'founder_count', 'top_5', 'top_20', 'top_50', 'top_5_bool',
       'top_20_bool', 'top_50_bool', 'n_female_founders', 'female_ratio',
       'mean_comp_founded_ever', 'mean_comp_founded_before', 'target'],
      dtype='object')

In [56]:
companies[companies.rounds_before_a!=0].drop(columns=['phd', 'MBA', 'cs', 'graduate', 'undergrad',
       'professional', 'degree_count', 'MBA_bool', 'cs_bool', 'phd_bool',
       'founder_count', 'top_5', 'top_20', 'top_50', 'top_5_bool',
       'top_20_bool', 'top_50_bool', 'n_female_founders', 'female_ratio',
       'mean_comp_founded_ever', 'mean_comp_founded_before', 'target'])

,id,normalized_name,category_code,founded_at,description,closed_at,exit,exit_date,country_code,state_code,date_series_a,participants_a,raised_amount_usd_a,timediff_founded_series_a,rounds_before_a,raised_before_a,participants_before_a
19,c:1972,salesforce,enterprise,1999-01-01,enterprise Enterprise Cloud Computing Salesf...,NaN,ipo,2004-07-02 00:00:00,USA,CA,1999-12-02,4,17000000.0,0.917199,1.0,4000000.0,4000000.0
20,c:59,google,search,1998-09-07,search Google is a search and advertising serv...,NaN,ipo,2004-08-25 00:00:00,USA,CA,1999-06-07,2,25000000.0,0.747449,1.0,100000.0,100000.0
25,c:1385,ask,web,1996-04-01,web Real answers instead of blue links Ask.c...,NaN,acquisition,2005-07-01 00:00:00,USA,CA,1999-03-01,2,25000000.0,2.913133,1.0,NaN,NaN
27,c:62,skype,software,2003-08-01,software VoIP Software Skype is a software ...,NaN,acquisition,2005-09-12 00:00:00,LUX,NaN,2003-08-01,1,0.0,0.000000,1.0,250000.0,250000.0
34,c:22291,metreos,software,2001-01-01,software IP Telephony Development Tools Metr...,NaN,acquisition,2006-06-28 00:00:00,USA,TX,2004-10-14,2,3800000.0,3.783787,1.0,555000.0,555000.0
38,c:5963,gate5,mobile,1999-01-01,mobile Gate5 offers mapping and location based...,NaN,acquisition,2006-09-01 00:00:00,DEU,NaN,2004-03-01,4,8114600.0,5.163693,1.0,NaN,NaN
48,c:5587,iconclude,software,2005-01-01,"software As of April 26, 2007, iConclude C...",NaN,acquisition,2007-03-07 00:00:00,USA,WA,2005-08-31,2,3000000.0,0.662573,1.0,500000.0,500000.0
73,c:86,zing systems,games_video,2007-06-01,"games_video ""In a world with ZING, you can...",NaN,acquisition,2007-08-07 00:00:00,USA,CA,2007-01-01,2,13000000.0,-0.413424,1.0,13000000.0,13000000.0
101,c:434,snocap,games_video,NaT,games_video Snocap is a digital rights and con...,NaN,acquisition,2008-02-01 00:00:00,USA,CA,2004-12-01,2,10000000.0,NaN,1.0,NaN,NaN
105,c:3531,pikum,games_video,2007-01-01,games_video Online betting game Backed by Fi...,2009-01-09,acquisition,2008-03-01 00:00:00,GBR,NaN,2008-03-03,2,5270000.0,1.169086,1.0,650000.0,650000.0


In [7]:
companies.mean_comp_founded_before.value_counts()

0.000000    3078
1.000000      79
0.500000      71
0.333333      44
0.250000      17
0.200000       9
2.000000       8
0.666667       5
1.500000       3
0.400000       2
0.800000       2
0.750000       1
4.000000       1
3.000000       1
0.600000       1
1.333333       1
0.571429       1
1.666667       1
0.375000       1
Name: mean_comp_founded_before, dtype: int64

In [10]:
exits ={"acquisition":1, "ipo":1, "no exit":0}
companies["target"] = companies.exit.map(exits)

In [26]:
companies.category_code.value_counts()

software            1740
web                 1056
biotech              669
mobile               481
games_video          446
enterprise           389
cleantech            379
advertising          355
hardware             304
semiconductor        284
ecommerce            281
public_relations     268
network_hosting      243
other                196
security             132
search               129
consulting            94
medical               73
analytics             69
finance               52
social                36
music                 31
news                  28
education             28
manufacturing         27
messaging             27
travel                20
health                19
transportation        12
automotive            12
nanotech              11
photo_video           10
real_estate           10
legal                 10
sports                 9
fashion                8
hospitality            7
nonprofit              4
local                  2
government             1
